# Разбивка на train и test

In [3]:
razbivka = transactions_train.groupby(['gender', 'client_id']).count().reset_index().set_index('client_id').sample(n=1250, random_state=42)
train = transactions_train[~transactions_train.index.isin(razbivka.index)]
test = transactions_train[transactions_train.index.isin(razbivka.index)]

In [4]:
test_target = test['gender']

test = test.drop('gender', axis=1)

In [6]:
dataset = pd.concat([train, test])

In [9]:
dataset_new = preprocessing_data(dataset)

In [13]:
test = dataset_new.query('gender.isna()')
train = dataset_new.query('~gender.isna()')